In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
concrete = pd.read_csv('data/concrete.csv')
co2 = pd.read_csv('data/co2.csv')

concrete['co2_lower'] = sum([concrete[col] * co2.loc[co2.ingredient == col, 'lower_bound'].values[0] for col in concrete.columns[:7]])
concrete['co2_upper'] = sum([concrete[col] * co2.loc[co2.ingredient == col, 'upper_bound'].values[0] for col in concrete.columns[:7]])

concrete = concrete[concrete['age'] < 120]

concrete_as = concrete[(concrete['ash'] > 0) & (concrete['slag'] > 0)]
concrete_a = concrete[(concrete['ash'] > 0) & (concrete['slag'] == 0)]
concrete_s = concrete[(concrete['ash'] == 0) & (concrete['slag'] > 0)]
concrete_ = concrete[(concrete['ash'] == 0) & (concrete['slag'] == 0)]

concrete_train, concrete_test = train_test_split(concrete,
                                                 shuffle=True,
                                                 random_state=487)

features = concrete.columns[:-3]

In [3]:
# This function was modified from stackexchange user hughdbrown 
# at this link, 
# https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset

# This returns the power set of a set minus the empty set
def powerset(s):
    power_set = []
    x = len(s)
    for i in range(1 << x):
        power_set.append([s[j] for j in range(x) if (i & (1 << j))])
        
    return power_set[1:]


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
def forest_grid(max_depths,features,n_trees,concrete_train):

    grid_cv = GridSearchCV(RandomForestRegressor(), # first put the model object here
                              param_grid = {'max_depth':max_depths, # place the grid values for max_depth and
                                            'n_estimators':n_trees}, # and n_estimators here
                              scoring = 'neg_mean_squared_error', # put the metric we are trying to optimize here as a string, "accuracy"
                              cv = 5) # put the number of cv splits here

    ## you fit it just like a model
    grid_cv.fit(concrete_train[features], concrete_train['strength'])

    return grid_cv.best_params_, -grid_cv.best_score_

In [ ]:
features2 = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg', 'exp_age']

max_depths = [5,6,7,8,9,10,11,12,13, 50]
features = ['cement', 'slag', 'water', 'superplastic', 'coarseagg', 'fineagg', 'exp_age']
n_trees = [100, 500]


for factor in np.arange(.01, .1, .01):
    concrete_train['exp_age'] = np.exp(-factor * concrete_train['age'])
    print(factor, forest_grid(max_depths,features,n_trees,concrete_train))

{'max_depth': 12, 'n_estimators': 100}
29.249699627404482
0.01 ({'max_depth': 12, 'n_estimators': 100}, 29.249699627404482)
{'max_depth': 12, 'n_estimators': 100}
29.522849057716957
0.02 ({'max_depth': 12, 'n_estimators': 100}, 29.522849057716957)
